<h3>Import Google colab to load the google drive</h3>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


<h3>Import Required Packages</h3>

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import os

<h3>Declare fido test videos and training images file path</h3>

In [3]:
fido_images_path = '/content/gdrive/My Drive/Capstone/Characters/FidoDido/Images/'
fido_videos_path = '/content/gdrive/My Drive/Capstone/Characters/FidoDido/Videos/'
os.listdir(fido_videos_path)

['FidoDidoV1.webm',
 'FidoDidoV2.webm',
 'FidoDidoV3.webm',
 'FidoDidoV4.webm',
 'FidoDidoV5.webm',
 'FidoDidoV6.webm',
 'FidoDidoV7.webm',
 'FidoDidoV8.webm',
 'VidImg',
 'FidoDidoV9.webm',
 'VidOutput']

<h3>Declare MickeyMouse test videos and training images file path</h3>

In [4]:
mm_images_path = '/content/gdrive/My Drive/Capstone/Characters/MickeyMouse/Images/'
mm_videos_path = '/content/gdrive/My Drive/Capstone/Characters/MickeyMouse/Videos/'
os.listdir(mm_videos_path)

['MMV1.mp4', 'MMV2.webm', 'VidImg', 'VidOutput']

<h3>Declare MrBean test videos and training images file path</h3>

In [5]:
mb_images_path = '/content/gdrive/My Drive/Capstone/Characters/MrBean/Images/'
mb_videos_path = '/content/gdrive/My Drive/Capstone/Characters/MrBean/Videos/'
os.listdir(mb_videos_path)

['BV1.webm', 'VidImg']

<h3>Declare FelixTheCat test videos and training images file path</h3>

In [6]:
fc_images_path = '/content/gdrive/My Drive/Capstone/Characters/Felix/Images/'
fc_videos_path = '/content/gdrive/My Drive/Capstone/Characters/Felix/Videos/'
os.listdir(fc_videos_path)

['y2mate.com - felix_the_cat_uncle_toms_crabbin_1920_1929_british_pathe_ZpjeGHUimIU_360p.mp4',
 'FC_V1.webm',
 'VidImg']

<h3>Custom function to create folder</h3>

In [0]:
import os

def createFolder(directory):# creates a folder in the location mentioned
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)
    

In [0]:
def deleteFile(file):# creates a folder in the location mentioned
    import os
    import shutil
    try:
        if(file.__contains__('.')):
          if(os.path.exists(file)):
            os.remove(file)
        else:
          if(os.path.exists(file)):
            shutil.rmtree(file)
    except OSError:
        print ('Error: Removing file. ' +  file + OSError.with_traceback())

In [0]:
deleteFile('/content/gdrive/My Drive/Capstone/Characters/MickeyMouse/Videos/VidOutput/MMV2/')

In [0]:
# Example
#createFolder(fc_videos_path+'VidImg/')
# Creates a folder in the current directory called data

In [0]:
import cv2
import math

<h3>Code to convert videos to image frames</h3>

<h2>FinalPredictionVideo</h2>

In [0]:
def convertVideoToImgFrames(videoPath, videoFile, imgFrameFolder):
  count = 0
  videoFileName = videoFile.split('.')[0]
  createFolder(videoPath+imgFrameFolder)
  createFolder(videoPath+imgFrameFolder+'/'+videoFileName+'/')
  cap = cv2.VideoCapture(videoPath+videoFile)   # capturing the video from the given path
  frameRate = cap.get(5) #frame rate
  x=1
  while(cap.isOpened()):
      frameId = cap.get(1) #current frame number
      ret, frame = cap.read()
      if (ret != True):
          break
      if (frameId % math.floor(frameRate) == 0):
          filename ="frame%d.jpg" % count;count+=1
          cv2.imwrite(videoPath+imgFrameFolder+'/'+videoFileName+'/'+filename, frame)
  cap.release()
  print ("Done!")

In [0]:
def loadModelAndWeights(modelJsonFile, modelWeightsFile):
  #  ---------------Load model and weights from disk-------------  #
  from keras.models import model_from_json

  # load json from saved location 

  json_file = open(modelJsonFile, 'r')
  loaded_model_json = json_file.read()
  json_file.close()

  # create model from json and load weights

  model = model_from_json(loaded_model_json)

  model.load_weights(modelWeightsFile)

  return model

In [0]:
# Compile the model
#model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [0]:
def preProcessVideoToImgFrmArray(videosPath, videoFile, imgFrameFolder, model_input_shape):
  videoFileName = videoFile.split('.')[0]
  vid_img_frame_path = videosPath+imgFrameFolder+'/'+videoFileName+'/'
  fd_frameX=[]
  for frame in os.listdir(vid_img_frame_path):
    fd_frameX.append(cv2.resize(cv2.imread(vid_img_frame_path+frame),model_input_shape)/255)
  return fd_frameX

In [0]:
def selectVidImgFrame(videos_path,video_file,imgFrameFolder, model, model_input_shape, characterToFind):
  classesToCharDic = {'fc':0,'fd':1,'mb':2,'mm':3}
  videoFileName = video_file.split('.')[0]
  vid_img_frame_path = videos_path+imgFrameFolder+'/'+videoFileName+'/'
  
  fd_frameX = preProcessVideoToImgFrmArray(videos_path,video_file, imgFrameFolder, model_input_shape)
  fd_frameX = np.array(fd_frameX)
  
  fd_frameY = model.predict(fd_frameX)
  #fd_frameY
  
  count = 0
  
  fd_pred_list=[]
  non_fd_pred_list=[]
  fd_X_list=[]
  non_fd_X_list=[]
  fd_X_imgName_Lst=[]
  non_fd_X_imgName_Lst=[]
  
  for fdFrame in fd_frameY:
    if(np.argmax(fdFrame) == classesToCharDic.get(characterToFind)): #check whether the model is predicting FidoDido
      fd_pred_list.append(count)
      fd_X_list.append(cv2.imread(vid_img_frame_path+'frame'+str(count)+'.jpg'))
      fd_X_imgName_Lst.append('frame'+str(count)+'.jpg')
    else:
      non_fd_pred_list.append(count)
      non_fd_X_list.append(cv2.imread(vid_img_frame_path+'frame'+str(count)+'.jpg'))
      non_fd_X_imgName_Lst.append('frame'+str(count)+'.jpg')
    count = count + 1
  return fd_X_list,fd_pred_list,non_fd_X_list,non_fd_pred_list,fd_X_imgName_Lst,non_fd_X_imgName_Lst

In [0]:
def copyFilesFromToRespective(videos_path,video_file, imgFrameFolder, X_imgNameLst, X_nonImgNameLst, vidOutputPath, imgVidOutputPath, nonImgVidOutputPath):
  videoFileName = video_file.split('.')[0]
  vid_output=videos_path+vidOutputPath+'/'+videoFileName+'/'
  deleteFile(vid_output)
  char_vid_output=vid_output+imgVidOutputPath+'/'
  non_char_vid_output=vid_output+nonImgVidOutputPath+'/'
  createFolder(vid_output)
  createFolder(char_vid_output)
  createFolder(non_char_vid_output)
  videoFileName = video_file.split('.')[0]
  vid_img_frame_path = videos_path+imgFrameFolder+'/'+videoFileName+'/'
  from shutil import copyfile
  for img in X_imgNameLst:
    copyfile(vid_img_frame_path+img,char_vid_output+img)
  for img in X_nonImgNameLst:
    copyfile(vid_img_frame_path+img,non_char_vid_output+img)
  return vid_output,char_vid_output,non_char_vid_output

In [0]:
def convert_frames_to_video(pathIn,pathOut,fps):
    import cv2
    import numpy as np
    import os
 
    from os.path import isfile, join
    import os
    #os.remove(pathOut)
    deleteFile(pathOut)
    frame_array = []
    files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]
    
    #for sorting the file names properly
    
    #files.sort()
    
    files.sort(key = lambda x: int((x[5:-4])))
 
    for i in range(len(files)):
        if(np.logical_not(files[i].endswith('.avi'))):
          filename=pathIn + files[i]
          #reading each files
          img = cv2.imread(filename)
          height, width, layers = img.shape
          size = (width,height)
          print(filename)
          #inserting the frames into an image array
          frame_array.append(img)
 
    out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
 
    for i in range(len(frame_array)):
        # writing to a image array
        out.write(frame_array[i])
    out.release()

In [0]:
def summarizeVideo(videos_path, video_file, imgFrameFolder, modelJsonFile, modelWeightsFile, model_input_shape, characterToFind, vidOutputPath, imgVidOutputPath, nonImgVidOutputPath):
  fps = 5.0
  convertVideoToImgFrames(videos_path, video_file, imgFrameFolder)
  loadedModel = loadModelAndWeights(modelJsonFile, modelWeightsFile)
  # Compile the model
  loadedModel.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
  loadedModel.summary()
  X_list,pred_list,non_X_list,non_pred_list,X_imgName_Lst,non_X_imgName_Lst = selectVidImgFrame(videos_path, video_file, imgFrameFolder, loadedModel, model_input_shape, characterToFind)
  vid_output,char_vid_output,non_char_vid_output = copyFilesFromToRespective(videos_path, video_file, imgFrameFolder, X_imgName_Lst, non_X_imgName_Lst, vidOutputPath, imgVidOutputPath, nonImgVidOutputPath)
  convert_frames_to_video(char_vid_output, char_vid_output+'summarized_char_video.avi', fps)

In [0]:
videos_path = fido_videos_path
video_file = 'FidoDidoV9.webm'
imgFrameFolder = 'VidImg'
modelFolder = '/content/gdrive/My Drive/Capstone/Characters/Model/'
modelJsonFile = modelFolder+'11Jun2019/vgg16/vgg16_model.json'
modelWeightsFile = modelFolder+'11Jun2019/vgg16/vgg16_best_weights.h5'
model_input_shape = (224,224)
characterToFind = 'fd'
vidOutputPath = 'VidOutput'
imgVidOutputPath = 'char'
nonImgVidOutputPath = 'nonChar'

summarizeVideo(videos_path, video_file, imgFrameFolder, modelJsonFile, modelWeightsFile, model_input_shape, characterToFind, vidOutputPath, imgVidOutputPath, nonImgVidOutputPath)

Done!


Using TensorFlow backend.
W0619 05:59:01.462994 140482094438272 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0619 05:59:01.507959 140482094438272 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0619 05:59:01.546352 140482094438272 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0619 05:59:01.700610 140482094438272 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:131: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0619 05:59:01.701568 140482

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [0]:
videos_path = mm_videos_path
video_file = 'MMV2.webm'
imgFrameFolder = 'VidImg'
modelFolder = '/content/gdrive/My Drive/Capstone/Characters/Model/'
modelJsonFile = modelFolder+'11Jun2019/vgg16/vgg16_model.json'
modelWeightsFile = modelFolder+'11Jun2019/vgg16/vgg16_best_weights.h5'
model_input_shape = (224,224)
characterToFind = 'mm'
vidOutputPath = 'VidOutput'
imgVidOutputPath = 'char'
nonImgVidOutputPath = 'nonChar'

summarizeVideo(videos_path, video_file, imgFrameFolder, modelJsonFile, modelWeightsFile, model_input_shape, characterToFind, vidOutputPath, imgVidOutputPath, nonImgVidOutputPath)

Done!
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
____